[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_bodies/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_bodies.sh http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.sh
! curl -o observation_prediction_bodies.py http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.py
! curl -o __init__.py http://josd.github.io/eye/transduction/transduction_bodies/__init__.py
! curl -o sample_bodies.observation http://josd.github.io/eye/transduction/transduction_bodies/sample_bodies.observation
! chmod +x observation_prediction_bodies.sh
% rm -fr ~/t2t_data/observation_prediction_bodies
% rm -fr ~/t2t_train/observation_prediction_bodies/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.2)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.6.7)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.3.0)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (19.8.1)
Requirement not upgraded as not directly required: gym in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.10.5)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: sympy in /usr/local/lib/python3.6/dist-packages (from tensor2

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # wind turbine size factor
      size_factor = 

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_bodies.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_bodies.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBL

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_bodies
INFO:tensorflow:Generating data for observation_prediction_bodies.
INFO:tensorflow:Generating vocab file: /content/t2t_data/observation_prediction_bodies/vocab.observation_prediction_bodies.16384.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 431
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 324
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 329
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 327
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iterati

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-14 23:02:19.413579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-14 23:02:19.414042: I tensorflow/core/common

INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:02:57
INFO:tensorflow:Saving dict for global step 100: global_step = 100, loss = 3.9433804, metrics-observation_prediction_bodies/accuracy = 0.036904763, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.0, metrics-observation_prediction_bodies/accuracy_top5 = 0.49266666, metrics-observation_prediction_bodies/approx_bleu_score = 0.32333392, metrics-observation_prediction_bodies/neg_log_perplexity = -4.7236314, metrics-observation_prediction_bodies/rouge_2_fscore = 0.5286281, metrics-observation_prediction_bodies/rouge_L_fscore = 0.74439347
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Us

INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:03:48
INFO:tensorflow:Saving dict for global step 200: global_step = 200, loss = 1.5114205, metrics-observation_prediction_bodies/accuracy = 0.48961905, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.0, metrics-observation_prediction_bodies/accuracy_top5 = 0.90104765, metrics-observation_prediction_bodies/approx_bleu_score = 0.55448717, metrics-observation_prediction_bodies/neg_log_perplexity = -1.89663, metrics-observation_prediction_bodies/rouge_2_fscore = 0.6959751, metrics-observation_prediction_bodies/rouge_L_fscore = 0.8678571
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INFO:tensorflow:Calling model_fn.
INF

INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:04:37
INFO:tensorflow:Saving dict for global step 300: global_step = 300, loss = 0.5349391, metrics-observation_prediction_bodies/accuracy = 0.90328574, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.25604397, metrics-observation_prediction_bodies/accuracy_top5 = 0.95842856, metrics-observation_prediction_bodies/approx_bleu_score = 0.698787, metrics-observation_prediction_bodies/neg_log_perplexity = -0.451369, metrics-observation_prediction_bodies/rouge_2_fscore = 0.7630612, metrics-observation_prediction_bodies/rouge_L_fscore = 0.8974376
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files:

...

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:15:32
INFO:tensorflow:Saving dict for global step 1600: global_step = 1600, loss = 0.050131254, metrics-observation_prediction_bodies/accuracy = 0.9885238, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.9117216, metrics-observation_prediction_bodies/accuracy_top5 = 0.9994762, metrics-observation_prediction_bodies/approx_bleu_score = 0.97654915, metrics-observation_prediction_bodies/neg_log_perplexity = -0.0944238, metrics-observation_prediction_bodies/rouge_2_fscore = 0.9773243, metrics-observation_prediction_bodies/rouge_L_fscore = 0.9900963
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:t

INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:16:22
INFO:tensorflow:Saving dict for global step 1700: global_step = 1700, loss = 0.044325583, metrics-observation_prediction_bodies/accuracy = 0.99, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.9230769, metrics-observation_prediction_bodies/accuracy_top5 = 0.99961907, metrics-observation_prediction_bodies/approx_bleu_score = 0.9793755, metrics-observation_prediction_bodies/neg_log_perplexity = -0.09074447, metrics-observation_prediction_bodies/rouge_2_fscore = 0.9801361, metrics-observation_prediction_bodies/rouge_L_fscore = 0.99132663
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:17:11
INFO:tensorflow:Saving dict for global step 1800: global_step = 1800, loss = 0.03973224, metrics-observation_prediction_bodies/accuracy = 0.99233335, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.9410256, metrics-observation_prediction_bodies/accuracy_top5 = 0.9997619, metrics-observation_prediction_bodies/approx_bleu_score = 0.9845999, metrics-observation_prediction_bodies/neg_log_perplexity = -0.089134045, metrics-observation_prediction_bodies/rouge_2_fscore = 0.9849887, metrics-observation_prediction_bodies/rouge_L_fscore = 0.993441
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:t

INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:18:01
INFO:tensorflow:Saving dict for global step 1900: global_step = 1900, loss = 0.03467261, metrics-observation_prediction_bodies/accuracy = 0.99471426, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.95934063, metrics-observation_prediction_bodies/accuracy_top5 = 0.9998095, metrics-observation_prediction_bodies/approx_bleu_score = 0.98927784, metrics-observation_prediction_bodies/neg_log_perplexity = -0.085673034, metrics-observation_prediction_bodies/rouge_2_fscore = 0.98959196, metrics-observation_prediction_bodies/rouge_L_fscore = 0.9954535
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INFO:tensorflow:Calling

INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-23:18:51
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.031181475, metrics-observation_prediction_bodies/accuracy = 0.99490476, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.96080583, metrics-observation_prediction_bodies/accuracy_top5 = 0.9998571, metrics-observation_prediction_bodies/approx_bleu_score = 0.98964185, metrics-observation_prediction_bodies/neg_log_perplexity = -0.08404767, metrics-observation_prediction_bodies/rouge_2_fscore = 0.98995465, metrics-observation_prediction_bodies/rouge_L_fscore = 0.99561226
INFO:tensorflow:Stop training model as max steps reached
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.float

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_bodies.observation
print("->-")
! pygmentize -g sample_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.35	A_PERSON has BMI class O	-3.34	A_PERSON has BMI class U	-4.86
A_TURBINE producing 9533 kW	-1.71	A_TURBINE producing 23832 kW	-1.97	A_TURBINE producing 16682 kW	-2.32
